### Notes:
- ingestion of SI dataset did not work straightforwardly: this dataset has 3D-geometry. The geoDB is not yet fit for this. It can easily be adapted, though, by simply dropping the 3rd dimension.
- the FR dataset seems broken, it shows weird shapes in QGIS and could not be ingested into the geoDB.
- I chose the datasets NRW, Belgium and Austria because of their large sizes
- joins over different dataset cannot be easily done with the geoDB, however, the data can be prepared (subset, filtering, ...) in the geoDB and then be finally processed (sum, average, ...) with geopandas.

In [ ]:
from xcube_geodb.core.geodb import GeoDBClient
import time

with open('credentials.py') as f:
    exec(f.read())
server_url = 'https://xcube-geodb.stage.brockmann-consult.de'  # this is enough so that geodb-stage database is used
server_port = 443
auth_domain = 'https://xcube-users.brockmann-consult.de/api/v2'
g = GeoDBClient(
    server_url=server_url,
    server_port=server_port,
    client_id=client_id,
    client_secret=client_secret,
    auth_aud=auth_domain
)

In [ ]:
tic = time.perf_counter()
srid_nrw = g.get_collection_srid('eurocrops-test-nrw')
bbox_nrw = g.get_collection_bbox('eurocrops-test-nrw')
srid_be = g.get_collection_srid('BE_VLG_2021_EC21')
bbox_be = g.get_collection_bbox('BE_VLG_2021_EC21')
srid_at = g.get_collection_srid('AT_2021_EC21')
bbox_at = g.get_collection_bbox('AT_2021_EC21')
bbox_nrw = g.transform_bbox_crs((bbox_nrw[1], bbox_nrw[0], bbox_nrw[3], bbox_nrw[2]), srid_nrw, 4326)
bbox_be = g.transform_bbox_crs((bbox_be[1], bbox_be[0], bbox_be[3], bbox_be[2]), srid_be, 4326)
bbox_at = g.transform_bbox_crs((bbox_at[1], bbox_at[0], bbox_at[3], bbox_at[2]), srid_at, 4326)
toc = time.perf_counter()
print(f"Computed the bounding boxes of NRW, BE (VLG), AT in WGS84 coords within {toc - tic:0.0f} seconds:")
print(f"NRW: {bbox_nrw}")
print(f"BE: {bbox_be}")
print(f"AT: {bbox_at}")

In [ ]:
tic = time.perf_counter()
df = g.get_collection('eurocrops-test-nrw', query='or=(code.eq.112,code.eq.115)')
toc = time.perf_counter()
area_acc = df['area_ha'].sum()
print(f"Computed the area growing wheat in NRW (Germany): {area_acc:0.0f} ha, within {toc - tic:0.0f} seconds")

In [ ]:
tic = time.perf_counter()
df = g.get_collection('BE_VLG_2021_EC21', query='or=(gwscod_h.eq.311,gwscod_h.eq.312)')
toc = time.perf_counter()
area_acc = df['graf_opp'].sum()
print(f"Computed the area growing wheat in Belgium (VLG): {area_acc:0.0f} ha, within {toc - tic:0.0f} seconds")

In [ ]:
tic = time.perf_counter()
df = g.get_collection('AT_2021_EC21', query='or=(snar_code.eq.140,snar_code.eq.168,snar_code.eq.167,snar_code.eq.153,snar_code.eq.529,snar_code.eq.119,snar_code.eq.169,snar_code.eq.138,snar_code.eq.166,snar_code.eq.164,snar_code.eq.165,snar_code.eq.161,snar_code.eq.160,snar_code.eq.170,snar_code.eq.137)')
toc = time.perf_counter()
area_acc = df['sl_flaeche'].sum()
print(f"Computed the area growing all kinds of wheat in Austria: {area_acc:0.0f} ha, within {toc - tic:0.0f} seconds")

In [ ]:
tic = time.perf_counter()
df = g.get_collection('AT_2021_EC21', query='and=(or(snar_code.eq.140,snar_code.eq.168,snar_code.eq.167,snar_code.eq.153,snar_code.eq.529,snar_code.eq.119,snar_code.eq.169,snar_code.eq.138,snar_code.eq.166,snar_code.eq.164,snar_code.eq.165,snar_code.eq.161,snar_code.eq.160,snar_code.eq.170,snar_code.eq.137),sl_flaeche.gt.1)', limit=1000)
toc = time.perf_counter()
print(f"Extracted 1000 areas in Austria growing wheat, larger than 1 ha, within {toc - tic:0.0f} seconds")

In [ ]:
tic = time.perf_counter()
df = g.get_collection('eurocrops-test-nrw', query='and=(code.eq.311,area_ha.gt.1)', limit=1000)
toc = time.perf_counter()
print(f"Extracted 1000 areas in NRW growing wheat, larger than 1 ha, within {toc - tic:0.0f} seconds")

In [ ]:
tic = time.perf_counter()
df = g.get_collection('BE_VLG_2021_EC21', query='and=(or(gwscod_h.eq.311,gwscod_h.eq.312),graf_opp.gt.1)', limit=1000)
toc = time.perf_counter()
print(f"Extracted 1000 areas in Belgium (VLG) growing wheat, larger than 1 ha, within {toc - tic:0.0f} seconds")